
# INFORME TECNICO DETALLADO - EXPLOTACION IDOR - MELI
# Prueba Tecnica Meli - 2025
# Andres Mateo Diaz Cañon

El presente informe documenta el análisis realizado sobre los registros de red proporcionados en el archivo “three_months_logs.csv”. El objetivo fue determinar la existencia de actividad anómala vinculada a una vulnerabilidad de tipo Insecure Direct Object Reference (IDOR) en el endpoint /invoices/search de www.mercadolibre.com.

> **Importante:** Cambia `CSV_FILE` si tu ruta es distinta, y ajusta `delimiter` si tu CSV usa `;`.


## 1) Requisitos y  Verificacion de complementos

Para el funcionamiento correcto del Notebook, se requiere la instalacion de los siguientes paquetes y librerias.



In [ ]:

# Requisitos básicos
!python -m pip install langchain duckdb openai llama-index pandas pyarrow
!pip install langchain langchain-openai
!pip install reportlab




## 2) Configuración de ruta y validación del archivo

- En este campo se debe colocar la ruta del log, de esta manera se valida su existencia y comprobación de los datos.

In [ ]:



from pathlib import Path

# AJUSTA ESTA RUTA A TU ARCHIVO REAL, ya que sin esta comprobación no es posible acceder a los datos.
CSV_PATH = Path("TU_RUTA_DEL_LOG")

print("Archivo existe?:", CSV_PATH.exists())
print("Ruta:", CSV_PATH)




## 3) Carga del archivo CSV para el analisis con DuckDB
- Se usa esta libreria para hacer el tratamiento y procesamiento de los datos basados en el log, ya que por su peso de 1.1 GB los motores regulares no pueden procesarlo de manera directa. Para ello procedemos en la carga general tomando una muestra de los primeros 100.000 registros. De esta manera podemos cargar los datos no en un UTF-8.

In [ ]:
# En este fragmento del codigo, se usa la libreria y funcionalidad Duckdb, la cual nos permite procesar los datos del log como si fuera una vista de base de datos.

import duckdb
from pathlib import Path

#  Ajusta a la ruta exacta del CSV (evita directorios archivos ocultos)
CSV_FILE = Path("TU_RUTA_DEL_LOG")

con = duckdb.connect()

# Lee el CSV como todo VARCHAR e ignora bytes inválidos (no UTF-8)
rel = con.read_csv(
    CSV_FILE.as_posix(),
    header=True,
    sample_size=100000,
    ignore_errors=True,   # salta bytes/filas con codificación mala
    all_varchar=True,     # evita inferencia de tipos
    delimiter=','
)

# Crea la vista 'logs' para tus queries SQL
rel.create_view('logs')

# Pruebas rápidas
con.sql("SELECT * FROM logs LIMIT 5").df()

con.sql("""
WITH t AS (
  SELECT try_strptime(timestamp, '%Y-%d-%mT%H:%M:%S')    AS ts
  FROM logs
  UNION ALL
  SELECT try_strptime(timestamp, '%Y-%d-%mT%H:%M')       AS ts
  FROM logs
)
SELECT
  strftime(ts, '%Y-%m') AS mes,      -- ejemplo: 2020-12
  COUNT(*)              AS total_peticiones
FROM t
WHERE ts IS NOT NULL
GROUP BY 1
ORDER BY 1;

""").df()




In [ ]:

# Estructura de columnas detectadas
con.sql("DESCRIBE logs").df()


## 4) Información general del log

En este apartado se realiza la consulta general del Log, de esta manera podemos analizar los datos y entender el contexto del escenario propuesto. 

- Para esta investigación fue necesario realizar diferentes cruces como tambien analisis de los datos, ya que al inicio se identifico cierta cantidad anomala de peticiones procedentes de diferentes sitios, dando a entender que el ataque podria tener diferentes naturalezas en comportamiento, es decir, explotaciones activas de IDOR con automatizaciones continuas sobre el API.

In [ ]:

# Total de peticiones
con.sql("SELECT COUNT(*) AS total_peticiones FROM logs").df()


In [ ]:

# Rango temporal
con.sql("""
SELECT MIN(timestamp) AS fecha_inicio, MAX(timestamp) AS fecha_fin
FROM logs
""").df()


In [ ]:

# Top 20 IPs
con.sql("""
SELECT source_ip, COUNT(*) AS total_peticiones
FROM logs
WHERE http_uri LIKE '%/invoices/search%'
GROUP BY source_ip
ORDER BY total_peticiones DESC
LIMIT 20
""").df()


In [ ]:

# Códigos HTTP (nota: columna es http_staus en el CSV)
con.sql("""
SELECT http_staus AS status, COUNT(*) AS cantidad
FROM logs
WHERE http_uri LIKE '%/invoices/search%'
GROUP BY http_staus
ORDER BY cantidad DESC
""").df()


In [ ]:

# Métodos
con.sql("""
SELECT http_method, COUNT(*) AS cantidad
FROM logs
WHERE http_uri LIKE '%/invoices/search%'
GROUP BY http_method
ORDER BY cantidad DESC
""").df()


In [ ]:

# Usuarios-agente (Top 20)
con.sql("""
SELECT user_agent, COUNT(*) AS cantidad
FROM logs
WHERE http_uri LIKE '%/invoices/search%'
GROUP BY user_agent
ORDER BY cantidad DESC
LIMIT 20
""").df()


In [ ]:

# Referers (Top 20)
con.sql("""
SELECT http_host, COUNT(*) AS cantidad
FROM logs
WHERE http_uri LIKE '%/invoices/search%'
GROUP BY http_host
ORDER BY cantidad DESC
LIMIT 20
""").df()


In [ ]:

# Serie por hora (0-23)
con.sql("""
SELECT strftime(timestamp, '%H') AS hora, COUNT(*) AS peticiones
FROM logs
WHERE http_uri LIKE '%/invoices/search%'
GROUP BY hora
ORDER BY hora
""").df()


In [ ]:
# Serie por pais

con.sql("""
SELECT
  CASE
    WHEN http_uri LIKE '%site_id=MeliAR%' THEN 'MeliAR - Mercado Libre Argentina'
    WHEN http_uri LIKE '%site_id=MeliBR%' THEN 'MeliBR - Mercado Libre Brasil'
    WHEN http_uri LIKE '%site_id=MeliMX%' THEN 'MeliMX - Mercado Libre México'
    WHEN http_uri LIKE '%site_id=MeliCL%' THEN 'MeliCL - Mercado Libre Chile'
    WHEN http_uri LIKE '%site_id=MeliCO%' THEN 'MeliCO - Mercado Libre Colombia'
    ELSE 'Otro / No especificado'
  END AS pais,
  COUNT(*) AS total_peticiones
FROM logs
WHERE http_uri LIKE '%site_id=Meli%'
GROUP BY 1
ORDER BY total_peticiones DESC;
""").df()



In [ ]:

# Muestras de invoice_id y tokens más consultados por esa IP
con.sql("""
WITH q AS (
  SELECT
    regexp_extract(http_uri, 'invoice_id=([^&]+)', 1) AS invoice_id,
    regexp_extract(http_uri, 'authtoken=([^&]+)', 1)  AS authtoken
  FROM logs
  WHERE http_uri LIKE '%/invoices/search%'
)
SELECT authtoken, COUNT(*) AS peticiones
FROM q
GROUP BY authtoken
ORDER BY peticiones DESC

""").df()


## 5) Análisis específico con Agente IA

In [ ]:
# === Normalizador automático de columnas → crea/actualiza vista `logs` ===
import pandas as pd

schema = con.sql("DESCRIBE rel").df()
cols_lower = {c.lower(): c for c in schema["column_name"].tolist()}

def resolve(*candidates):
    for c in candidates:
        if c in cols_lower:
            return cols_lower[c]
    return None  # no existe

# Detecta columnas reales
c_timestamp   = resolve("timestamp", "time", "ts", "datetime", "date")
c_http_host   = resolve("http_host", "host", "domain")
c_http_uri    = resolve("http_uri", "uri", "path", "request_uri", "url")
c_http_status = resolve("http_status", "http_staus", "status", "status_code", "code")
c_http_method = resolve("http_method", "method", "verb")
c_user_agent  = resolve("user_agent", "http_user_agent", "ua")
c_client_ip   = resolve("client_ip", "source_ip", "ip", "remote_addr", "x_forwarded_for")

def sel(col, alias):
    # si existe, úsalo; si no, crea NULL::VARCHAR con alias para no romper
    return f"{col} AS {alias}" if col else f"NULL::VARCHAR AS {alias}"

select_sql = f"""
CREATE OR REPLACE VIEW logs AS
SELECT
  {sel(c_timestamp,   'timestamp')},
  {sel(c_http_host,   'http_host')},
  {sel(c_http_uri,    'http_uri')},
  {sel(c_http_status, 'http_status')},
  {sel(c_http_method, 'http_method')},
  {sel(c_user_agent,  'user_agent')},
  {sel(c_client_ip,   'client_ip')}
FROM rel
"""

con.sql(select_sql)

print("✅ Vista `logs` creada/actualizada con columnas estandarizadas:")
print(con.sql("DESCRIBE logs").df().to_string(index=False))


## IMPORTANTE ESTABLECER API KEY

In [ ]:
# 2) Establecimiento API KEY
import os
os.environ["OPENAI_API_KEY"] = "TU_API_KEY_AQUI"

- Para la creacion de este script, se utilizo langchain como fuente principal para el tratamiento de las LLM y poder generar las consultas a traves de OpenIA

In [ ]:
# ============================================================
# El siguiente script permite realizar las siguientes acciones sobre la informacion cargada:
# con análisis completo de logs + Agente IA + PDF con tablas
#  y agente con herramienta SQL consultando el 100% del log)
# ============================================================

import os
import re
import duckdb
import pandas as pd
from datetime import datetime

# ========== Configuración ==========
CSV_FILE   = r"TU_RUTA_LOG_AQUI"  # Ruta confirmada
OUTPUT_PDF = "informe_forense_completo.pdf"
SAMPLE_MAX = 200   # Muestra opcional solo para contexto; el LLM puede consultar TODO con sql_agg

# ========== Conexión DuckDB ==========
try:
    con  # si ya existe en el entorno interactivo
except NameError:
    con = duckdb.connect(database=":memory:")

# ========== Helpers robustos ==========
def table_exists(conn, name: str) -> bool:
    """Verifica existencia de tabla usando information_schema (sin lanzar excepción)."""
    q = """
    SELECT COUNT(*) AS n
    FROM information_schema.tables
    WHERE table_schema IN ('main','temp') AND table_name = ?
    """
    return conn.execute(q, [name]).fetchone()[0] > 0

def reload_rel_from_csv_streaming(conn, csv_path: str) -> None:
    """
    Crea/actualiza la tabla 'rel' leyendo el CSV en modo 'streaming' de DuckDB.
    No carga todo en memoria; procesa directamente desde el archivo.
    """
    if not os.path.exists(csv_path):
        raise RuntimeError(f"No existe el CSV en la ruta: {csv_path}")
    print(f"📥 (Streaming) Construyendo tabla 'rel' desde: {csv_path}")
    conn.execute(f"""
        CREATE OR REPLACE TABLE rel AS
        SELECT * FROM read_csv_auto('{csv_path}', HEADER=TRUE);
    """)
    print("✅ Tabla 'rel' creada/actualizada (streaming)")

def quoted_or_null(col: str) -> str:
    return f'"{col}"' if col else "NULL"

def df_or_empty(sql: str) -> pd.DataFrame:
    try:
        return con.execute(sql).df()
    except Exception:
        return pd.DataFrame()

def df_to_markdown_chunks(df: pd.DataFrame, chunk_size: int = 50, title: str = "Tabla"):
    if df is None or df.empty:
        return [f"### {title}\n_No hay datos disponibles._\n"]
    chunks = []
    for i in range(0, len(df), chunk_size):
        sub = df.iloc[i:i+chunk_size]
        header = f"### {title} (filas {i+1}-{min(i+chunk_size, len(df))} de {len(df)})"
        chunks.append(header + "\n" + sub.to_markdown(index=False) + "\n")
    return chunks

def markdown_or_note(df: pd.DataFrame, title: str) -> str:
    if df is None or df.empty:
        return f"### {title}\n_No hay datos disponibles._\n"
    return f"### {title}\n" + df.to_markdown(index=False) + "\n"

# ========== Asegurar 'rel' ==========
if not table_exists(con, "rel"):
    reload_rel_from_csv_streaming(con, CSV_FILE)
else:
    print("✅ La tabla 'rel' ya existe ")

# Doble verificación por si la carga falló
if not table_exists(con, "rel"):
    raise RuntimeError(
        "No se encontró la tabla 'rel' tras intentar cargar el CSV. "
        "Verifica la ruta, permisos y que el CSV no esté vacío/corrupto."
    )

# ========== Vista 'logs' segura con timestamp robusto ==========
# Detectar columnas reales presentes
cols_df = con.execute("""
SELECT column_name AS name
FROM information_schema.columns
WHERE table_schema IN ('main','temp') AND table_name = 'rel'
ORDER BY ordinal_position
""").df()

raw_cols = cols_df["name"].tolist()
lowmap   = {c.lower(): c for c in raw_cols}
def get_col(*cands):
    for c in cands:
        x = lowmap.get(c.lower())
        if x: return x
    return None

c_timestamp   = get_col("timestamp","time","datetime","date")
c_http_host   = get_col("http_host","host","server_name","authority")
c_http_uri    = get_col("http_uri","request_uri","uri","path","request")
c_http_status = get_col("http_status","status","code","http_staus")   # tolera typo
c_http_method = get_col("http_method","method","verb")
c_user_agent  = get_col("http_user_agent","user_agent","ua")
c_client_ip   = get_col("client_ip","source_ip","remote_addr","ip","src_ip")

concat_expr = " || ' ' || ".join([f'CAST("{c}" AS VARCHAR)' for c in raw_cols]) if raw_cols else "''"
q_ts = quoted_or_null(c_timestamp)

# Normalización de timestamp (AAAA-DD-MMTHH:MM → AAAA-MM-DDTHH:MM; + patrones alternos)
create_logs_sql = f"""
CREATE OR REPLACE VIEW logs AS
WITH base AS (
  SELECT
    COALESCE(
      -- 1) Formato declarado: AAAA-DD-MMTHH:MM(:SS opcional)
      try_strptime({q_ts}, '%Y-%d-%mT%H:%M:%S'),
      try_strptime({q_ts}, '%Y-%d-%mT%H:%M'),
      -- 2) Reordenamos a AAAA-MM-DDTHH:MM(:SS) y parseamos estándar
      try_strptime(
        REGEXP_REPLACE({q_ts}, '^(\\d{{4}})-(\\d{{2}})-(\\d{{2}})T', '\\1-\\3-\\2T'),
        '%Y-%m-%dT%H:%M:%S'
      ),
      try_strptime(
        REGEXP_REPLACE({q_ts}, '^(\\d{{4}})-(\\d{{2}})-(\\d{{2}})T', '\\1-\\3-\\2T'),
        '%Y-%m-%dT%H:%M'
      ),
      -- 3) Otras variantes posibles (ISO, con Z, con espacio)
      try_strptime({q_ts}, '%Y-%m-%dT%H:%M:%S'),
      try_strptime({q_ts}, '%Y-%m-%d %H:%M:%S'),
      try_strptime({q_ts}, '%Y-%m-%d'),
      TRY_CAST({q_ts} AS TIMESTAMP)
    ) AS timestamp,
    {quoted_or_null(c_http_host)}   AS http_host,
    {quoted_or_null(c_http_uri)}    AS http_uri,
    {quoted_or_null(c_http_status)} AS http_status,
    {quoted_or_null(c_http_method)} AS http_method,
    {quoted_or_null(c_user_agent)}  AS user_agent,
    {quoted_or_null(c_client_ip)}   AS client_ip,
    REGEXP_EXTRACT({concat_expr}, '(?:\\d{{1,3}}\\.){{3}}\\d{{1,3}}', 0) AS derived_ip
  FROM rel
)
SELECT
  timestamp,
  http_host,
  http_uri,
  http_status,
  http_method,
  user_agent,
  COALESCE(client_ip, derived_ip) AS client_ip
FROM base;
"""
con.execute(create_logs_sql)
print("✅ Vista 'logs'")

# ========== Datos clave de investigación (100% del log) ==========
df_timeline = df_or_empty("""
SELECT CAST(timestamp AS DATE) AS fecha, COUNT(*) AS peticiones
FROM logs
WHERE timestamp IS NOT NULL
GROUP BY 1
ORDER BY 1
""")

df_status = df_or_empty("""
SELECT http_status AS status, COUNT(*) AS total
FROM logs
GROUP BY 1
ORDER BY total DESC
""")

df_endpoints = df_or_empty("""
SELECT http_uri AS endpoint, COUNT(*) AS peticiones
FROM logs
GROUP BY 1
ORDER BY peticiones DESC
LIMIT 100
""")

df_ua = df_or_empty("""
SELECT user_agent, COUNT(*) AS total
FROM logs
GROUP BY 1
ORDER BY total DESC
LIMIT 25
""")

# IPs (evidencia resumida)
df_ip = df_or_empty("""
SELECT client_ip AS ip, COUNT(*) AS total
FROM logs
WHERE client_ip IS NOT NULL AND client_ip <> ''
GROUP BY 1
ORDER BY total DESC
LIMIT 100
""")

# TODAS las IPs (para CSV; no se insertará en el PDF)
df_ip_full = df_or_empty("""
SELECT client_ip AS ip, COUNT(*) AS total
FROM logs
WHERE client_ip IS NOT NULL AND client_ip <> ''
GROUP BY 1
ORDER BY total DESC
""")
df_ip_full.to_csv("ips_conteo_completo.csv", index=False, encoding="utf-8")
print("💾 Exportado CSV: ips_conteo_completo.csv")

# Heurística IDOR
df_idor = df_or_empty("""
SELECT http_uri AS endpoint, COUNT(*) AS hits
FROM logs
WHERE http_uri ~ '(\\?|&)(id|uid|user|invoice|order|doc|account|acc)='
   OR http_uri ~ '/\\d+'
GROUP BY 1
ORDER BY hits DESC
LIMIT 500
""")

# Authtoken más consultados (global) por /invoices/search
df_authtoken_global = con.execute("""
WITH q AS (
  SELECT
    regexp_extract(http_uri, 'invoice_id=([^&]+)', 1) AS invoice_id,
    regexp_extract(http_uri, 'authtoken=([^&]+)', 1)  AS authtoken
  FROM logs
  WHERE http_uri LIKE '%/invoices/search%'
)
SELECT authtoken, COUNT(*) AS peticiones
FROM q
GROUP BY authtoken
ORDER BY peticiones DESC
""").df()
df_authtoken_global.to_csv("authtoken_global.csv", index=False, encoding="utf-8")
print("💾 Exportado CSV: authtoken_global.csv")

# Volumen de peticiones por país (site_id)
df_por_pais = df_or_empty("""
SELECT
  CASE
    WHEN http_uri LIKE '%site_id=MeliAR%' THEN 'MeliAR - Mercado Libre Argentina'
    WHEN http_uri LIKE '%site_id=MeliBR%' THEN 'MeliBR - Mercado Libre Brasil'
    WHEN http_uri LIKE '%site_id=MeliMX%' THEN 'MeliMX - Mercado Libre México'
    WHEN http_uri LIKE '%site_id=MeliCL%' THEN 'MeliCL - Mercado Libre Chile'
    WHEN http_uri LIKE '%site_id=MeliCO%' THEN 'MeliCO - Mercado Libre Colombia'
    ELSE 'Otro / No identificado'
  END AS site_id,
  COUNT(*) AS total_peticiones
FROM logs
GROUP BY 1
ORDER BY total_peticiones DESC
""")
df_por_pais.to_csv("site_id_resumen.csv", index=False, encoding="utf-8")
print("💾 Exportado CSV: site_id_resumen.csv")

# ========== Evidencia Markdown para IA ==========
evidencia_md = "\n".join([
    "*(Todas las tablas de esta sección se calcularon sobre el **100%** del log)*\n",
    markdown_or_note(df_timeline,   "Timeline diario"),
    markdown_or_note(df_por_pais,   "Volumen de peticiones por país (site_id)"),
    markdown_or_note(df_status,     "Distribución de códigos de estado"),
    markdown_or_note(df_endpoints,  "Endpoints más solicitados"),
    markdown_or_note(df_ua,         "User-Agents más frecuentes"),
    markdown_or_note(df_ip,         "Direcciones IP con mayor volumen (top 100)"),
    markdown_or_note(df_authtoken_global.head(50), "Authtoken más consultados — /invoices/search (top 50)"),
    markdown_or_note(df_idor,       "Patrones posibles de IDOR (heurística)"),
])

# Muestra pequeña opcional (el LLM PUEDE IGNORARLA; tiene acceso a sql_agg para 100%)
df_logs_full = df_or_empty("SELECT * FROM logs")
if df_logs_full.empty:
    sample_md = "_El conjunto de logs está vacío._"
else:
    sample_n = min(len(df_logs_full), SAMPLE_MAX)
    df_sample = df_logs_full.sample(sample_n, random_state=42) if len(df_logs_full) > sample_n else df_logs_full
    sample_md = df_sample.head(SAMPLE_MAX).to_markdown(index=False)

# ========== Agente IA (LangChain) — con herramienta SQL sobre el 100% ==========
respuesta_ia = None
ia_error = None
try:
    from langchain_openai import ChatOpenAI
    from langchain.agents import initialize_agent, AgentType
    from langchain.tools import tool

    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
    if not OPENAI_API_KEY:
        raise RuntimeError(
            "No se encontró OPENAI_API_KEY en el entorno. "
            "Configura la variable de entorno antes de ejecutar este script."
        )

    # Herramienta segura para consultas SELECT sobre TODO el log.
    @tool("sql_agg", return_direct=False)
    def sql_agg(sql: str) -> str:
        """
        Ejecuta consultas SQL **solo-lectura (SELECT)** sobre la base completa (100% del log).
        Devuelve el resultado en Markdown. Limita a 2000 filas para no exceder el tamaño.
        Seguridad:
          - Rechaza consultas que no comiencen por 'SELECT' (ignorando espacios/lineas).
          - Rechaza palabras peligrosas: CREATE/INSERT/UPDATE/DELETE/ALTER/DROP/REPLACE/ATTACH/COPY.
        """
        if not isinstance(sql, str) or not sql.strip():
            return "⚠️ sql_agg: Debes enviar una consulta SELECT no vacía."
        sql_clean = sql.strip().lstrip("(\n\t ").upper()
        if not sql_clean.startswith("SELECT"):
            return "⛔ sql_agg: Solo se permiten consultas SELECT."
        forbidden = ("CREATE","INSERT","UPDATE","DELETE","ALTER","DROP","REPLACE","ATTACH","COPY","PRAGMA","IMPORT","EXPORT","LOAD")
        if any(w in sql_clean for w in forbidden):
            return "⛔ sql_agg: Consulta rechazada por contener palabras no permitidas."
        try:
            df = con.execute(sql).df()
            if df.empty:
                return "_(sin filas)_"
            # Limitar filas para respuesta
            if len(df) > 2000:
                df = df.head(2000)
                note = "\n_(resultado truncado a 2000 filas)_\n"
            else:
                note = ""
            return df.to_markdown(index=False) + note
        except Exception as e:
            return f" Error ejecutando SQL: {e}"

    @tool("get_aggregates_md", return_direct=False)
    def get_aggregates_md(_: str = "") -> str:
        """Devuelve las tablas agregadas (100% del log) en Markdown."""
        return evidencia_md

    @tool("get_sample_md", return_direct=False)
    def get_sample_md(_: str = "") -> str:
        """Devuelve una muestra representativa pequeña del log en Markdown (opcional)."""
        return sample_md

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [sql_agg, get_aggregates_md, get_sample_md]

    try:
        agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.OPENAI_FUNCTIONS,
            verbose=False
        )
    except TypeError:
        agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent_type=AgentType.OPENAI_FUNCTIONS,
            verbose=False
        )

    prompt = f"""
Eres un analista forense experto. Todo el **contexto agregado** proviene del **100% del log**.
Si necesitas verificar cualquier detalle, puedes usar la herramienta **sql_agg** para ejecutar
consultas SQL *SELECT* directamente sobre el conjunto de datos completo (100% del log).

## Evidencia agregada (100% del log; tablas)
{evidencia_md}

## Muestra opcional (solo referencia rápida; puedes ignorarla si usas sql_agg)
{sample_md}

Con base en TODO el log (usa preferentemente los agregados del 100% y, si hace falta,
consulta con sql_agg), elabora un **informe forense completo en Markdown** con la siguiente estructura y formato:

## Resumen ejecutivo
- Contexto del incidente, alcance e impacto potencial, partiendo del LOG completo. Indica la vulnerabilidad predominante si la evidencias.

## Narrativa del incidente
- Línea de tiempo de eventos relevantes y fases del ataque (usa timeline y/o agrégalo con sql_agg si requieres granularidad).

## Indicadores de compromiso (IoC)
- Direcciones IP, endpoints, user-agents y comportamientos anómalos. Presenta **tablas** cuando corresponda.

## Posibles patrones IDOR
- Evidencias de manipulación de parámetros y accesos indebidos (usa **tablas** y justifica por qué podrían ser IDOR).

## Estadísticas clave del incidente
- **Tablas Markdown** con la distribución de status, endpoints más atacados, volumen por IP, site_id/país, etc.

## Análisis de User-Agents
- Destaca bots o automatizaciones (wget, curl, python, scrapy, spider, bot) con **tablas**.

## Recomendaciones
- Acciones inmediatas y mitigaciones a mediano y largo plazo, detalladas.

### Reglas estrictas de formato
- Usa **títulos H2/H3** consistentes.
- Donde presentes datos, usa **tablas Markdown** (| col1 | col2 | ... |).
- **NO RESUMAS** filas de tablas: si una tabla es larga, divídela en bloques de 50 filas, **sin omitir filas**.
- No incluyas trazas ni llamadas a herramientas en la respuesta (solo resultados).
"""
    respuesta_ia = agent.run(prompt)
    print("Respuesta IA (vista previa):\n", (respuesta_ia or "")[:1500])

except Exception as e:
    ia_error = repr(e)
    print("ℹ️ No se pudo ejecutar el agente IA:", ia_error)
    partes = [
        "# Informe Forense (sin IA)",
        "## Resumen",
        "No se pudo ejecutar el modelo. Se incluyen agregados para referencia (100% del log).",
        evidencia_md
    ]
    respuesta_ia = "\n\n".join(partes)

# ========== Sección fija adicional en el PDF ==========
# 1) Authtoken completos (en bloques para no truncar)
authtoken_chunks = df_to_markdown_chunks(
    df_authtoken_global,
    chunk_size=50,
    title="Authtoken más consultados — /invoices/search (completo)"
)
seccion_authtoken = "\n".join(authtoken_chunks)

# 2) Volumen de peticiones por país (site_id) — sección solicitada
site_id_chunks = df_to_markdown_chunks(
    df_por_pais,
    chunk_size=50,
    title="Volumen de peticiones por país (site_id)"
)
seccion_site_id = "\n".join(site_id_chunks)

# Unir secciones fijas al informe final
respuesta_ia = (
    (respuesta_ia or "# Informe Forense")
    + "\n\n## Sección fija — Volumen de peticiones por país (site_id)\n"
    + seccion_site_id
    + "\n\n> Archivo complementario: `site_id_resumen.csv`\n"
    + "\n\n## Sección fija — Authtoken más consultados\n"
    + seccion_authtoken
    + "\n\n> Archivo complementario: `authtoken_global.csv`"
)

# ========== Conversión Markdown -> PDF (ReportLab con auto-wrap de celdas) ==========
try:
    from reportlab.lib.pagesizes import A4
    from reportlab.lib import colors
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, ListFlowable, ListItem
    from reportlab.lib.units import cm

    styles = getSampleStyleSheet()
    STY_H1 = ParagraphStyle("H1", parent=styles["Heading1"], fontSize=16, leading=20, spaceAfter=10)
    STY_H2 = ParagraphStyle("H2", parent=styles["Heading2"], fontSize=14, leading=18, spaceAfter=8)
    STY_H3 = ParagraphStyle("H3", parent=styles["Heading3"], fontSize=12, leading=16, spaceAfter=6)
    STY_BODY = ParagraphStyle("BODY", parent=styles["BodyText"], fontSize=10.0, leading=13.5, spaceAfter=6)

    def parse_md_tables(md_text: str):
        """Devuelve lista de bloques: {'type': 'table'|'heading'|'list'|'p', ...}"""
        lines = md_text.splitlines()
        blocks, buf_para = [], []

        def flush_para():
            if buf_para:
                blocks.append({"type":"p", "text":"\n".join(buf_para).strip()})
                buf_para.clear()

        i = 0
        while i < len(lines):
            line = lines[i].rstrip()

            # Encabezados
            if re.match(r"^#{1,6}\s+", line):
                flush_para()
                level = len(line) - len(line.lstrip("#"))
                text = line[level:].strip()
                blocks.append({"type":"heading", "level":level, "text":text})
                i += 1
                continue

            # Listas
            if re.match(r"^\s*[-*]\s+", line):
                flush_para()
                items = []
                while i < len(lines) and re.match(r"^\s*[-*]\s+", lines[i]):
                    items.append(re.sub(r"^\s*[-*]\s+", "", lines[i]).strip())
                    i += 1
                blocks.append({"type":"list", "items":items})
                continue

            # Tablas Markdown
            if '|' in line:
                if i+1 < len(lines) and re.match(r'^\s*\|?[:\-| ]+\|?\s*$', lines[i+1]):
                    tbl_lines = [line, lines[i+1]]
                    i += 2
                    while i < len(lines) and '|' in lines[i] and lines[i].strip():
                        tbl_lines.append(lines[i])
                        i += 1
                    rows = []
                    for idx, tline in enumerate(tbl_lines):
                        parts = [c.strip() for c in tline.strip().strip('|').split('|')]
                        if idx == 1:
                            continue  # separador
                        rows.append(parts)
                    if rows:
                        flush_para()
                        blocks.append({"type":"table", "rows":rows})
                    continue

            # Párrafos
            if line.strip() == "":
                flush_para()
            else:
                buf_para.append(line)
            i += 1

        flush_para()
        return blocks

    # ====== AJUSTE DE TABLAS: ancho máximo + auto-wrap en celdas ======
    from reportlab.platypus import Table, TableStyle, Paragraph
    from reportlab.lib.units import cm

    def md_to_story(md_text: str):
        blocks = parse_md_tables(md_text or "")
        story = []
        # Portada
        story.append(Paragraph("Informe Forense Automatizado", STY_H1))
        story.append(Paragraph(datetime.now().strftime("%Y-%m-%d %H:%M"), STY_BODY))
        story.append(Spacer(1, 0.5*cm))

        for b in blocks:
            if b["type"] == "heading":
                sty = STY_H1 if b["level"] <= 1 else STY_H2 if b["level"] == 2 else STY_H3
                story.append(Paragraph(b["text"], sty))

            elif b["type"] == "list":
                items = [ListItem(Paragraph(it.replace("\n","<br/>"), STY_BODY)) for it in b["items"]]
                story.append(ListFlowable(items, bulletType="bullet"))

            elif b["type"] == "table":
                data = b["rows"]

                # Limitar ancho máximo de tabla: A4(21cm) - márgenes (2+2) = 17cm
                max_table_width = 17 * cm
                num_cols = max(len(data[0]), 1)
                col_width = max_table_width / num_cols

                # Envolver texto en celdas largas con saltos de línea
                wrapped_data = []
                for row in data:
                    wrapped_row = []
                    for cell in row:
                        text = str(cell)
                        # Si la celda es muy larga, inserta saltos cada ~60 chars
                        if len(text) > 60:
                            text = "<br/>".join([text[i:i+60] for i in range(0, len(text), 60)])
                        wrapped_row.append(Paragraph(text, STY_BODY))
                    wrapped_data.append(wrapped_row)

                t = Table(wrapped_data, colWidths=[col_width]*num_cols, repeatRows=1)
                t.setStyle(TableStyle([
                    ('FONTNAME',(0,0),(-1,-1),'Helvetica'),
                    ('FONTSIZE',(0,0),(-1,-1),8.5),
                    ('BACKGROUND',(0,0),(-1,0),colors.HexColor("#F0F2F5")),
                    ('TEXTCOLOR',(0,0),(-1,0),colors.black),
                    ('GRID',(0,0),(-1,-1),0.3,colors.HexColor("#B8C1CC")),
                    ('ALIGN',(0,0),(-1,0),'CENTER'),
                    ('ALIGN',(0,1),(-1,-1),'LEFT'),
                    ('VALIGN',(0,0),(-1,-1),'TOP'),
                    ('ROWBACKGROUNDS',(0,1),(-1,-1),[colors.white, colors.HexColor("#FCFEFF")]),
                    ('TOPPADDING',(0,0),(-1,-1),3),
                    ('BOTTOMPADDING',(0,0),(-1,-1),3),
                ]))
                story.append(t)

            else:  # párrafo
                text = b["text"].replace("&", "&amp;").replace("<","&lt;").replace(">","&gt;")
                story.append(Paragraph(text.replace("\n","<br/>"), STY_BODY))

            story.append(Spacer(1, 0.25*cm))
        return story

    from reportlab.platypus import SimpleDocTemplate
    from reportlab.lib.pagesizes import A4

    doc = SimpleDocTemplate(
        OUTPUT_PDF, pagesize=A4,
        topMargin=1.5*cm, bottomMargin=1.5*cm, leftMargin=2*cm, rightMargin=2*cm
    )
    story = md_to_story(respuesta_ia or "# Informe vacío")
    doc.build(story)
    print(f"✅ PDF generado: {os.path.abspath(OUTPUT_PDF)}")
    print("CSVs generados: ips_conteo_completo.csv, authtoken_global.csv, site_id_resumen.csv")
    if ia_error:
        print(" Nota: el PDF se generó, pero la IA falló y se usó contenido de respaldo.")

except ModuleNotFoundError as e:
    print("Falta una librería para exportar PDF:", e)
    print("Instala con:  pip install reportlab")


## 6) ALMACENADOR DE MD PARA USO DE CHAT INTERACTIVO.


In [ ]:
# ================================================
# Guardar el informe en Markdown, es necesario este paso para poder usar el el chat interactivo, este se genera a traves del uso de la anterior celda.
# - Usa la salida real del agente (respuesta_ia) si existe
# - Si no, guarda un respaldo con la evidencia agregada
# - Normaliza saltos de línea, asegura UTF-8 y crea una versión con timestamp
# ================================================
import os
import sys
import io
from datetime import datetime

def _safe_text(s) -> str:
    """Convierte a str, normaliza saltos de línea y asegura fin de archivo con newline."""
    if s is None:
        return ""
    if not isinstance(s, str):
        s = str(s)
    # normaliza CRLF/CR a LF
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    # quita null bytes por si vinieran de alguna conversión extraña
    s = s.replace("\x00", "")
    # asegura 1 salto de línea final
    if not s.endswith("\n"):
        s += "\n"
    return s

def _build_fallback_md() -> str:
    partes = [
        "# Informe Forense (respaldo)\n",
        f"_Generado automáticamente: {datetime.now().strftime('%Y-%m-%d %H:%M')}_\n",
        "## Nota\nNo se obtuvo salida del modelo IA. Se incluyen agregados para referencia.\n",
    ]
    ev = globals().get("evidencia_md")
    if isinstance(ev, str) and ev.strip():
        partes.append(ev)
    else:
        partes.append("_No hay evidencia agregada disponible._")
    return "\n\n".join(partes)

# 1) Determinar el contenido a guardar
contenido_md = None
resp = globals().get("respuesta_ia")
if isinstance(resp, str) and resp.strip():
    contenido_md = resp
else:
    contenido_md = _build_fallback_md()

contenido_md = _safe_text(contenido_md)

# 2) Construir nombre de salida
#    - base: informe_forense_completo.md (compatible con tu flujo)
#    - copia con timestamp para trazabilidad
base_name = "informe_forense_completo"
OUTPUT_MD = f"{base_name}.md"

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_MD_TS = f"{base_name}_{ts}.md"

# 3) (Opcional) Inyectar metadatos mínimos al inicio si no existen encabezados
if not contenido_md.lstrip().startswith("# "):
    meta = [
        f"# Informe Forense Automatizado",
        f"_Generado: {datetime.now().strftime('%Y-%m-%d %H:%M')}_",
        "",
    ]
    contenido_md = _safe_text("\n".join(meta) + contenido_md)

# 4) Escribir a disco (UTF-8) en dos archivos: el “canonical” y la copia con timestamp
def _write_utf8(path: str, text: str):
    # Usa io.open para asegurar comportamiento consistente en distintos entornos
    with io.open(path, "w", encoding="utf-8", newline="\n") as f:
        f.write(text)

try:
    _write_utf8(OUTPUT_MD, contenido_md)
    _write_utf8(OUTPUT_MD_TS, contenido_md)

    # 5) Mensajes de confirmación útiles
    abs_main = os.path.abspath(OUTPUT_MD)
    abs_ts   = os.path.abspath(OUTPUT_MD_TS)

    size_main = os.path.getsize(OUTPUT_MD)
    size_ts   = os.path.getsize(OUTPUT_MD_TS)

    print("✅ Informe Markdown guardado correctamente.")
    print(f"   • Principal : {abs_main}  ({size_main:,} bytes)")
    print(f"   • Timestamp : {abs_ts}  ({size_ts:,} bytes)")

    # 6) Consejos de integración con tu flujo actual
    #    - Si usas el chat forense interactivo, éste buscará 'informe_forense_completo.md' por defecto.
    #    - La copia con timestamp te permite versionar rápidamente entregables.

except Exception as e:
    # Falla segura con explicación
    print("⛔ Error al guardar el informe en Markdown:", e, file=sys.stderr)
    print("   Verifica permisos de escritura y rutas.", file=sys.stderr)


## 7) CHAT INTERACTIVO

In [ ]:
# ================================================================
# Chat
# - Carga robusta del informe Markdown
# - QA con LLM usando el informe como contexto (recorte inteligente)
# - Búsqueda de fragmentos relevantes en el informe
# - Presets de preguntas forenses frecuentes
# ================================================================

import os, re, textwrap, pathlib
from datetime import datetime
from typing import List

# ---------- 1) Localización robusta del informe ----------
def _find_latest_report(base_name="informe_forense_completo", exts=(".md",)):
    """Devuelve la ruta del informe más reciente."""
    candidates = []
    cwd = pathlib.Path(".")
    for ext in exts:
        candidates += list(cwd.glob(f"{base_name}{ext}"))
        candidates += list(cwd.glob(f"{base_name}_*.{ext.strip('.')}"))
    if not candidates:
        return None
    candidates = sorted(candidates, key=lambda p: p.stat().st_mtime, reverse=True)
    return str(candidates[0])

if "out_md" in globals() and isinstance(out_md, str) and os.path.exists(out_md):
    INFORME_PATH = out_md
else:
    INFORME_PATH = _find_latest_report() or "informe_forense_completo.md"

if not os.path.exists(INFORME_PATH):
    raise FileNotFoundError(
        f"No encontré el informe: {INFORME_PATH}\n"
        "Asegúrate de haber generado el Markdown o ajusta INFORME_PATH."
    )

with open(INFORME_PATH, "r", encoding="utf-8", errors="ignore") as f:
    INFORME_MD = f.read()

# ---------- 2) Utilidades de contexto ----------
def _trim_context_sections(md: str, max_chars: int = 120_000) -> str:
    """Recorte inteligente manteniendo las secciones más relevantes."""
    md = md.replace("\r\n", "\n").replace("\r", "\n")
    if len(md) <= max_chars:
        return md

    parts = re.split(r'(?m)^#{2,3}\s+', md)
    headers = re.findall(r'(?m)^#{2,3}\s+(.+)$', md)
    sections = []
    lead = parts[0]
    for h, body in zip(headers, parts[1:]):
        sections.append((h.strip(), body))

    signals = re.compile(
        r'\b(401|403|404|500|SQL|IDOR|token|authtoken|invoice|/invoices|login|auth|csrf|xss|timeout|error|bot|curl|wget|python|spider|scrapy|endpoint|site_id|ip|user-agent)\b',
        re.I
    )
    scored = []
    for h, body in sections:
        score = len(signals.findall(body)) + len(signals.findall(h))
        scored.append((score, h, body))
    scored.sort(key=lambda x: x[0], reverse=True)

    keep = []
    total = len(lead)
    keep.append(lead)
    i = 0
    while i < len(scored) and total < max_chars * 0.85:
        score, h, body = scored[i]
        chunk = f"\n\n## {h}\n{body}"
        total += len(chunk)
        keep.append(chunk)
        i += 1

    tail = md[-min(len(md)//6, int(max_chars*0.15)):]
    keep.append("\n\n...[contexto recortado]...\n\n")
    keep.append(tail)
    return "".join(keep)

CTX_FULL = INFORME_MD
CTX = _trim_context_sections(INFORME_MD)

def _context_snippets(query: str, md: str, window=220, max_hits=6) -> List[str]:
    """Extrae fragmentos relevantes del informe."""
    if not query or not query.strip():
        return []
    toks = [re.escape(t) for t in re.findall(r'\w{3,}', query.lower())]
    if not toks:
        return []
    pat = re.compile("|".join(toks), re.I)
    hits = []
    for m in pat.finditer(md):
        start = max(0, m.start() - window)
        end = min(len(md), m.end() + window)
        frag = md[start:end].replace("\n", " ")
        hits.append("… " + frag + " …")
        if len(hits) >= max_hits:
            break
    return hits

# ---------- 3) LLM (LangChain + OpenAI) ----------
LLM_READY = True
try:
    from langchain_openai import ChatOpenAI
    from langchain.prompts import ChatPromptTemplate
except Exception as e:
    LLM_READY = False
    _IMPORT_ERR = repr(e)

if LLM_READY:
    try:
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    except Exception as e:
        LLM_READY = False
        _INIT_ERR = repr(e)

# ---------- 4) Diálogo forense ----------
def dialogo_forense(pregunta: str, incluir_snippets: bool = True) -> str:
    """Pregunta a la IA usando el informe forense como contexto."""
    if not pregunta or not pregunta.strip():
        return "Por favor escribe una pregunta."

    snippets = _context_snippets(pregunta, CTX_FULL) if incluir_snippets else []

    if not LLM_READY:
        detalles = []
        if "_IMPORT_ERR" in globals(): detalles.append(f"ImportError: {_IMPORT_ERR}")
        if "_INIT_ERR" in globals():   detalles.append(f"InitError: {_INIT_ERR}")
        extra = ("\n\nDetalles: " + " | ".join(detalles)) if detalles else ""
        base = (
            "⚠️ El modelo de IA no está disponible. "
            "Verifica tu API Key y que `langchain-openai` esté instalado."
            + extra
        )
        if snippets:
            base += "\n\n### Fragmentos relevantes\n" + "\n\n".join([f"> {s}" for s in snippets])
        return base

    instruct = (
        "Eres un analista forense experto. Responde en español, claro y accionable. "
        "Si algo no está en el contexto, dilo explícitamente y sugiere qué dato falta."
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", instruct),
        ("system", f"Contexto del informe (Markdown recortado):\n{CTX}"),
        ("human", "{question}"),
    ])
    try:
        msg = prompt.format_messages(question=pregunta)
        ans = llm.invoke(msg)
        out = ans.content if hasattr(ans, "content") else str(ans)

        if snippets:
            out += "\n\n### Fragmentos relevantes\n" + "\n\n".join([f"> {s}" for s in snippets])
        return out
    except Exception as e:
        base = (
            "⚠️ No pude obtener respuesta del modelo.\n\n"
            f"Error: {e}\n\n"
            "Sugerencias: prueba con una pregunta más corta o revisa los límites de tokens."
        )
        if snippets:
            base += "\n\n### Fragmentos relevantes\n" + "\n\n".join([f"> {s}" for s in snippets])
        return base

# ---------- 5) UI con ipywidgets ----------
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

PRESETS = [
    "Resumen del incidente",
    "Principales indicadores de compromiso",
    "Patrones IDOR detectados",
    "Recomendaciones inmediatas",
    "Análisis de User-Agents sospechosos",
    "Fases del ataque en la línea de tiempo",
]

inp_q = widgets.Textarea(
    placeholder='Haz una pregunta sobre la investigación forense…',
    description='Pregunta:',
    layout=widgets.Layout(width='100%', height='80px')
)
chk_snip = widgets.Checkbox(value=True, description="Incluir fragmentos del informe")
ddl_preset = widgets.Dropdown(options=PRESETS, description="Preguntas rápidas:")
btn = widgets.Button(description="Preguntar", button_style='primary', icon="search")
out = widgets.Output(layout={'border': '1px solid #ddd'})

def _on_preset_change(change):
    if change['name'] == 'value':
        inp_q.value = f"Explícame: {change['new']}"

ddl_preset.observe(_on_preset_change, names='value')

def _on_click(_):
    with out:
        clear_output()
        print("🧠 Respuesta:")
        resp = dialogo_forense(inp_q.value, incluir_snippets=chk_snip.value)
        display(Markdown(resp))

btn.on_click(_on_click)

display(widgets.VBox([
    inp_q,
    chk_snip,
    ddl_preset,
    btn,
    widgets.HTML("<b>💬 Respuesta:</b>"),
    out
]))

print(f"📄 Informe cargado: {os.path.abspath(INFORME_PATH)}")
print(f"🪪 LLM listo: {LLM_READY}")


## GRACIAS POR LA ATENCION PRESTADA!!!

- **NOTA:El notebook, esta organizado para que sea ejecutado celda por celda, de esta manera se garantiza la linealidad del funcionamiento del codigo.**